# Segmentation of tomato discrepancies and diseases

### Setup

Install required libraries

In [ ]:
!pip install -e .

In [ ]:
!pip install onnx

Import required libraries

In [ ]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
import os
import glob

from ultralytics import YOLO
from ultralytics import settings

settings.update({"wandb": False})

from IPython.display import display, Image

In [ ]:
# Define number of epochs for training and image size
SAVE_PERIOD = 35
CLOSE_MOSAIC = 10
EPOCHS = 1
IMAGE_SIZE = 640

EXPERIMENTS_DIR = "C:\\Users\\admin\\PycharmProjects\\tomato_discrepancies_project"
# Define directories and pathes to be used
EXPERIMENTS_FOLDER = f"{EXPERIMENTS_DIR}/tomato-discrepancies-segmentation"

DATASET_FOLDER = f"{EXPERIMENTS_FOLDER}/datasets/TomatoDiscrepanciesSegmentation"

os.makedirs(DATASET_FOLDER, exist_ok=True)

# MODEL_NAME_TO_USE = "yolo11n-seg" # yolov8n-seg
MODEL_NAME_TO_USE = "yolov8-all-c2f-simam-fixed-fully-adjusted-edges-simam-seg"
EXT_TYPE = "yaml" # pt or yaml; depends on whether you gonna use pretrained or from scratch
MODEL_TO_USE = f"{MODEL_NAME_TO_USE}.{EXT_TYPE}"

RUNS_FOLDER = "runs/tomato_discrepancies_segment"
EXPERIMENT_NAME = f"BASE_COCO_{MODEL_NAME_TO_USE}_{EPOCHS}_epochs"

#
TRAIN_EXPERIMENT_NAME = f"{EXPERIMENT_NAME}/train"
TRAINED_MODEL_PATH = f"{DATASET_FOLDER}/{RUNS_FOLDER}/{TRAIN_EXPERIMENT_NAME}/weights/best.pt"

#
VAL_EXPERIMENT_NAME = f"{EXPERIMENT_NAME}/val"
TEST_EXPERIMENT_NAME = f"{EXPERIMENT_NAME}/test"

# Define variables to the experiment folders where predictions will be stored
INFERENCE_TRAIN_EXPERIMENT_NAME = f"{TRAIN_EXPERIMENT_NAME}_predict"
INFERENCE_VAL_EXPERIMENT_NAME = f"{VAL_EXPERIMENT_NAME}_predict"
INFERENCE_TEST_EXPERIMENT_NAME = f"{TEST_EXPERIMENT_NAME}_predict"

# Define path variables to predicted images that you saved
TRAIN_PREDICTIONS_PATH = f"{DATASET_FOLDER}/{RUNS_FOLDER}/{INFERENCE_TRAIN_EXPERIMENT_NAME}"
VAL_PREDICTIONS_PATH = f"{DATASET_FOLDER}/{RUNS_FOLDER}/{INFERENCE_VAL_EXPERIMENT_NAME}"
TEST_PREDICTIONS_PATH = f"{DATASET_FOLDER}/{RUNS_FOLDER}/{INFERENCE_TEST_EXPERIMENT_NAME}"

# Define format and path for model export
FORMAT = "onnx"
EXPORT_EXPERIMENT_NAME = f"{EXPERIMENT_NAME}/{FORMAT}"

### 💻 CLI

In [ ]:
%cd "{DATASET_FOLDER}"

# Insert your own code for dataset downloading

### Train YOLOv8

Train the model and evaluate its performance metrics on train subset to gain insights into its effectiveness

In [ ]:
%cd {DATASET_FOLDER}

!yolo task=segment mode=train model="{MODEL_TO_USE}" data="{DATASET_FOLDER}/data.yaml" epochs="{EPOCHS}" imgsz="{IMAGE_SIZE}" plots=True project="{RUNS_FOLDER}" name="{TRAIN_EXPERIMENT_NAME}" save_period="{SAVE_PERIOD}" close_mosaic="{CLOSE_MOSAIC}"

### Calculate metrics for validation and test subsets

Calculate metrics for `train` subset.

In [ ]:
%cd {DATASET_FOLDER}

!yolo task=segment mode=val model="{TRAINED_MODEL_PATH}" data="{DATASET_FOLDER}/train_data.yaml" project="{RUNS_FOLDER}" name="{TRAIN_EXPERIMENT_NAME}/train_metrics"

Assess metrics on `val` subset

In [ ]:
%cd {DATASET_FOLDER}

!yolo task=segment mode=val model="{TRAINED_MODEL_PATH}" data="{DATASET_FOLDER}/data.yaml" project="{RUNS_FOLDER}" name="{VAL_EXPERIMENT_NAME}"

Assess metrics on `test` subset

In [ ]:
%cd {DATASET_FOLDER}

!yolo task=segment mode=val model="{TRAINED_MODEL_PATH}" data="{DATASET_FOLDER}/test_data.yaml" project="{RUNS_FOLDER}" name="{TEST_EXPERIMENT_NAME}"

### Visualize metrics

In [ ]:
Image(filename=f"{DATASET_FOLDER}/{RUNS_FOLDER}/{TRAIN_EXPERIMENT_NAME}/results.png", width=600)

In [ ]:
Image(filename=f"{DATASET_FOLDER}/{RUNS_FOLDER}/{TRAIN_EXPERIMENT_NAME}/val_batch0_pred.jpg", width=600)

In [ ]:
Image(filename=f"{DATASET_FOLDER}/{RUNS_FOLDER}/{TRAIN_EXPERIMENT_NAME}/train_metrics/confusion_matrix_normalized.png", width=650)

In [ ]:
Image(filename=f"{DATASET_FOLDER}/{RUNS_FOLDER}/{VAL_EXPERIMENT_NAME}/confusion_matrix_normalized.png", width=650)

In [ ]:
Image(filename=f"{DATASET_FOLDER}/{RUNS_FOLDER}/{TEST_EXPERIMENT_NAME}/confusion_matrix_normalized.png", width=650)

### Inference

Make predictions with your previously trained model.

In [ ]:
import yaml

def get_value_from_yaml(yaml_file, key):
    with open(yaml_file, 'r') as file:
        data = yaml.safe_load(file)
        value = data.get(key, "")
        return value

parent_path = get_value_from_yaml(f"{DATASET_FOLDER}/data.yaml", "path")

train_path = get_value_from_yaml(f"{DATASET_FOLDER}/data.yaml", "train")
val_path = get_value_from_yaml(f"{DATASET_FOLDER}/data.yaml", "val")
test_path = get_value_from_yaml(f"{DATASET_FOLDER}/data.yaml", "test")

train_source = os.path.join(parent_path, train_path)
val_source = os.path.join(parent_path, val_path)
test_source = os.path.join(parent_path, test_path)

print("Retrieved pathes")
print(f"Train: {train_source}")
print(f"Val: {val_source}")
print(f"Test: {test_source}")

In [ ]:
%cd {DATASET_FOLDER}

!yolo task=segment mode=predict model="{TRAINED_MODEL_PATH}" conf=0.25 source="{train_source}" save=True project="{RUNS_FOLDER}" name="{INFERENCE_TRAIN_EXPERIMENT_NAME}"

In [ ]:
%cd {DATASET_FOLDER}

!yolo task=segment mode=predict model="{TRAINED_MODEL_PATH}" conf=0.25 source="{val_source}" save=True project="{RUNS_FOLDER}" name="{INFERENCE_VAL_EXPERIMENT_NAME}"

In [ ]:
%cd {DATASET_FOLDER}

!yolo task=segment mode=predict model="{TRAINED_MODEL_PATH}" conf=0.25 source="{test_source}" save=True project="{RUNS_FOLDER}" name="{INFERENCE_TEST_EXPERIMENT_NAME}"

**NOTE:** Let's take a look at few results.

In [ ]:
def display_detected(path_to_pred_dir, predictions_to_display=5, display_width=600, return_display_list=False):
  image_path_list = []
  for image_path in glob.glob(f'{path_to_pred_dir}/*.jpg')[:predictions_to_display]:
      display(Image(filename=image_path, width=display_width))
      print("\n")
      if return_display_list:
        image_path_list.append(image_path)

  if return_display_list:
      return image_path_list

In [ ]:
display_detected(f"{TRAIN_PREDICTIONS_PATH}")

In [ ]:
display_detected(display_detected(f"{VAL_PREDICTIONS_PATH}"))

In [ ]:
test_images_inference_list = display_detected(display_detected(f"{TEST_PREDICTIONS_PATH}"), return_display_list=True)

In [ ]:
%cd {DATASET_FOLDER}

!yolo export model="{TRAINED_MODEL_PATH}" imgsz="{IMAGE_SIZE}" format="{FORMAT}" project="{RUNS_FOLDER}" name="{EXPORT_EXPERIMENT_NAME}" opset=18